In [45]:
import tkinter as tk
from tkinter import messagebox
import csv
import logging

In [46]:
# Configure logging for clear, minimal warnings
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(message)s')

def load_data():
    users = {}
    feedback_data = []
    report_data = []

    # Define the essential fields for each row type
    required_fields = {
        "user": {"type", "username", "password", "role"},
        "feedback": {"type", "feedback_id", "agent", "feedback_text", "status"},
        "report": {"type", "report_id", "date", "agent", "feedback_applied"}
    }
    
    # Set up the path to the CSV file
    csv_path = "data.csv"
    
    try:
        # Open and read the CSV file
        with open(csv_path, mode="r") as file:
            reader = csv.DictReader(file)
            
            # Check if the file is empty (no headers or data)
            if reader.fieldnames is None:
                logging.warning("The CSV file is empty or missing headers.")
                return users, feedback_data, report_data

            # Process each row based on its type
            for row in reader:
                row_type = row.get("type")
                
                # Check if the row type is one of the expected types
                if row_type in required_fields:
                    # Validate required fields based on the row type
                    missing_fields = required_fields[row_type] - row.keys()
                    
                    # Skip row if required fields are missing
                    if missing_fields:
                        logging.warning(f"Skipping {row_type} row with missing fields: {row}")
                        continue
                    
                    # Populate data based on row type
                    if row_type == "user":
                        users[row["username"]] = {
                            "password": row["password"],
                            "role": row["role"]
                        }
                    elif row_type == "feedback":
                        feedback_data.append({
                            "feedback_id": row["feedback_id"],
                            "agent": row["agent"],
                            "feedback_text": row["feedback_text"],
                            "status": row["status"],
                            "feedback_entry_date": row.get("feedback_entry_date", ""),
                            "integration_date": row.get("integration_date", "")
                        })
                    elif row_type == "report":
                        report_data.append({
                            "report_id": row["report_id"],
                            "date": row["date"],
                            "agent": row["agent"],
                            "feedback_applied": row["feedback_applied"]
                        })
                else:
                    logging.warning(f"Unknown type '{row_type}' in row: {row}")

    except FileNotFoundError:
        logging.error(f"CSV file '{csv_path}' not found.")
    except IOError:
        logging.error(f"Error reading the file '{csv_path}'.")

    return users, feedback_data, report_data


In [47]:
def save_data(users, feedback_data, report_data, csv_path="data.csv"):
    # Define the order of the columns for consistency
    fieldnames = [
        "type", "username", "password", "role", 
        "feedback_id", "agent", "feedback_text", "status", 
        "report_id", "date", "feedback_applied", 
        "feedback_entry_date", "integration_date"
    ]
    
    # Prepare data for writing to CSV
    data_to_write = []
    
    # Convert users dictionary to list of dictionaries with "user" type
    for username, user_info in users.items():
        data_to_write.append({
            "type": "user",
            "username": username,
            "password": user_info["password"],
            "role": user_info["role"],
            "feedback_id": "",
            "agent": "",
            "feedback_text": "",
            "status": "",
            "report_id": "",
            "date": "",
            "feedback_applied": "",
            "feedback_entry_date": "",
            "integration_date": ""
        })
    
    # Add feedback entries to data list with "feedback" type
    for feedback in feedback_data:
        data_to_write.append({
            "type": "feedback",
            "username": "",
            "password": "",
            "role": "",
            "feedback_id": feedback["feedback_id"],
            "agent": feedback["agent"],
            "feedback_text": feedback["feedback_text"],
            "status": feedback["status"],
            "report_id": "",
            "date": "",
            "feedback_applied": "",
            "feedback_entry_date": feedback.get("feedback_entry_date", ""),
            "integration_date": feedback.get("integration_date", "")
        })
    
    # Add report entries to data list with "report" type
    for report in report_data:
        data_to_write.append({
            "type": "report",
            "username": "",
            "password": "",
            "role": "",
            "feedback_id": "",
            "agent": report["agent"],
            "feedback_text": "",
            "status": "",
            "report_id": report["report_id"],
            "date": report["date"],
            "feedback_applied": report["feedback_applied"],
            "feedback_entry_date": "",
            "integration_date": ""
        })
    
    # Write all data to CSV
    with open(csv_path, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data_to_write)


In [48]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Agent and Coach App")
        self.geometry("400x300")

        # Load data once and store it in attributes
        self.users, self.feedback_data, self.report_data = load_data()
        
        # Container frame to hold all screens
        container = tk.Frame(self)
        container.pack(fill="both", expand=True)
        
        # Dictionary to store frames
        self.frames = {}
        
        # Add each screen frame to the dictionary
        for F in (LoginScreen, AgentMainMenu, CoachMainMenu, ProfileSettings, ViewFeedback):
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        # Start with the login screen
        self.show_frame(LoginScreen)

    def show_frame(self, screen_class):
        """Bring a frame to the front based on the screen class."""
        frame = self.frames[screen_class]
        frame.tkraise()

In [49]:
class LoginScreen(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Login Screen").pack(pady=10)
        
        # Role selection
        self.role = tk.StringVar(value="Agent")
        tk.Radiobutton(self, text="Agent", variable=self.role, value="Agent").pack()
        tk.Radiobutton(self, text="Coach", variable=self.role, value="Coach").pack()
        
        # Username and Password
        tk.Label(self, text="Username").pack()
        self.username_entry = tk.Entry(self)
        self.username_entry.pack()

        tk.Label(self, text="Password").pack()
        self.password_entry = tk.Entry(self, show="*")
        self.password_entry.pack()
        
        # Login button
        login_button = tk.Button(self, text="Login", command=self.login)
        login_button.pack()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        role = self.role.get()

        # Validate login with the stored users data
        if username in self.controller.users:
            user = self.controller.users[username]
            if user["password"] == password and user["role"] == role:
                if role == "Agent":
                    self.controller.show_frame(AgentMainMenu)
                elif role == "Coach":
                    self.controller.show_frame(CoachMainMenu)
            else:
                messagebox.showerror("Login Error", "Incorrect password or role.")
        else:
            messagebox.showerror("Login Error", "Username not found.")

In [50]:
class AgentMainMenu(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Agent Main Menu").pack(pady=10)
        
        # Profile and View Feedback buttons
        profile_button = tk.Button(self, text="Profile", command=lambda: controller.show_frame(ProfileSettings))
        profile_button.pack()

        view_feedback_button = tk.Button(self, text="View Feedback", command=lambda: controller.show_frame(ViewFeedback))
        view_feedback_button.pack()

        logout_button = tk.Button(self, text="Logout", command=lambda: controller.show_frame(LoginScreen))
        logout_button.pack()

In [51]:
class CoachMainMenu(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Coach Main Menu").pack(pady=10)
        
        # Create Account, Input Feedback, Generate Reports buttons
        create_account_button = tk.Button(self, text="Create Account", command=self.create_account)
        create_account_button.pack()

        input_feedback_button = tk.Button(self, text="Input Feedback", command=self.input_feedback)
        input_feedback_button.pack()

        generate_reports_button = tk.Button(self, text="Generate Reports", command=self.generate_reports)
        generate_reports_button.pack()

        logout_button = tk.Button(self, text="Logout", command=lambda: controller.show_frame(LoginScreen))
        logout_button.pack()

    def create_account(self):
        # Placeholder for Create Account functionality
        messagebox.showinfo("Create Account", "Create Account screen (not implemented).")
    
    def input_feedback(self):
        # Placeholder for Input Feedback functionality
        messagebox.showinfo("Input Feedback", "Input Feedback screen (not implemented).")

    def generate_reports(self):
        # Placeholder for Generate Reports functionality
        messagebox.showinfo("Generate Reports", "Generate Reports screen (not implemented).")

In [52]:
class ProfileSettings(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Profile Settings").pack(pady=10)

        # Password fields
        tk.Label(self, text="Current Password").pack()
        self.current_password = tk.Entry(self, show="*")
        self.current_password.pack()

        tk.Label(self, text="New Password").pack()
        self.new_password = tk.Entry(self, show="*")
        self.new_password.pack()

        save_button = tk.Button(self, text="Save Changes", command=self.update_password)
        save_button.pack()

        back_button = tk.Button(self, text="Back to Main Menu", command=lambda: controller.show_frame(AgentMainMenu))
        back_button.pack()

    def update_password(self):
        # Placeholder for updating the password in the CSV
        messagebox.showinfo("Profile", "Password updated (simulated).")

In [53]:
class ViewFeedback(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="View Feedback").pack(pady=10)

        # Dropdown for feedback status
        self.status_var = tk.StringVar(value="Not Integrated")
        status_dropdown = tk.OptionMenu(self, self.status_var, "Not Integrated", "Integrated")
        status_dropdown.pack()

        refresh_button = tk.Button(self, text="Refresh Feedback", command=self.refresh_feedback)
        refresh_button.pack()

        back_button = tk.Button(self, text="Back to Main Menu", command=lambda: controller.show_frame(AgentMainMenu))
        back_button.pack()

    def refresh_feedback(self):
        # Placeholder for refreshing feedback from the stored feedback data
        messagebox.showinfo("Feedback", "Feedback refreshed (simulated).")

In [54]:
if __name__ == "__main__":
    app = App()
    app.mainloop()